# RUNNING LAMMPS ON GOOGLE COLAB WITH GPU
*Created by Ezugwu Isaac* @ejikeboniface042@gmail.com

Activate GPU

*Go to Edit > Notebook Settings > Hardware accelerator > GPU*


Mount drive of google colab




In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Installing LAMMPS

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Check System Specification

In [ ]:
%%bash
#Default for free account: 1GPU and 2 CPU with 1TB HDD and 12GB RAM
lscpu
nvidia-smi

Adding prebuild LAMMPS and openkim repository by [@anton gladky](https://launchpad.net/~gladky-anton/+archive/ubuntu/lammps)

In [ ]:
%%bash
sudo add-apt-repository ppa:gladky-anton/lammps
sudo add-apt-repository ppa:openkim/latest
sudo apt update

Installing LAMMPS, DOC, DATA and OPENKIM MODELS

In [ ]:
%%bash
sudo apt-get install lammps-stable lammps-stable-doc lammps-stable-data openkim-models

##inputing files

In [ ]:
!wget  https://www.ctcms.nist.gov/potentials/Download/1999--Liu-X-Y-Liu-C-L-Borucki-L-J--Al-Cu/2/al-cu-set.eam.alloy

In [38]:
%%writefile Al10Cu_Tensile.lmp
#-------Of course I shall document my tensile test of Al 10%wt Cu
#-----Yami_Senpai {-_-} 20th Mar 2022!!!!!! 

# ------------------------ INITIATION -------------------------------- 
units metal 
dimension 3 
boundary p p p 
atom_style atomic  
# ----------------------- Geometry of Material ----------------------- 
lattice fcc 4.05 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1
region whole block 0 10 0 10 0 10 
create_box 1 whole 
create_atoms 1 region whole
# --------------------Potential function-------------------------------- 
pair_style eam/alloy
pair_coeff * * al-cu-set.eam.alloy Al
#--------------------Equilibration-------------------------------------- 
compute csym all centro/atom fcc 
compute peratom all pe/atom 
reset_timestep 0 
timestep 0.001 
velocity all create 300 12345 mom yes rot no 
fix 1 all npt temp 300 300 1 iso 0 0 1 drag 1 
minimize 1.0e-4 1.0e-6 100 1000 
thermo 1000 
thermo_style custom step lx ly lz press pxx pyy pzz pe temp 
run 20000 
unfix 1 
dump ten all custom 500 dump.ten.* x y z
variable tmp equal "lx" 
variable L0 equal ${tmp} 
print "Initial Length, L0: ${L0}" 
#---------------------Stress-Strain Calculation---------------- 
reset_timestep 0 
fix 1 all npt temp 300 300 1 y 0 0 1 z 0 0 1 drag 1 
variable srate equal 1.0e10 
variable srate1 equal "v_srate / 1.0e12" 
fix 2 all deform 1 x erate ${srate1} units box remap x 
variable strain equal "(lx - v_L0)/v_L0" 
variable p1 equal "v_strain" 
variable p2 equal "-pxx/10000" 
variable p3 equal "-pyy/10000" 
variable p4 equal "-pzz/10000" 
fix def1 all print 100 "${p1} ${p2} ${p3} ${p4}" file Al10CuOutput.txt screen no 
dump 1 all custom 250 *.dump type x y z 
thermo 1000 
thermo_style custom step v_strain temp v_p2 v_p3 v_p4 ke pe press 
run 20000 
#---------------------simulation completely done------------ 
print "All done Zac!"

Overwriting Al10Cu_Tensile.lmp


##Run your lammps script

In [39]:
%%bash
mpirun --allow-run-as-root -n 1 lmp_stable -sf gpu -pk gpu 1 neigh no newton off -in Al10Cu_Tensile.lmp

LAMMPS (3 Mar 2020)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:94)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 4.05 4.05 4.05
Created orthogonal box = (0 0 0) to (40.5 40.5 40.5)
  1 by 1 by 1 MPI processor grid
Created 4000 atoms
  create_atoms CPU = 0.000495999 secs

--------------------------------------------------------------------------
- Using acceleration for eam/alloy:
-  with 1 proc(s) per device.
-  with 1 thread(s) per proc.
-  with OpenCL Parameters for: GENERIC
--------------------------------------------------------------------------
Device 0: Tesla T4, 40 CUs, 15 GB, 1.6 GHZ (Mixed Precision)
--------------------------------------------------------------------------

Initializing Device and compiling on process 0...Done.
Initializing Device 0 on core 0...Done.

Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 7.68